In [1]:
import pandas as pd
import numpy as np

from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer

from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder

In [2]:
pd.options.display.max_rows = 1000

In [5]:
df0 = pd.read_excel('train_data_2023.xlsx').tail(100000)

In [8]:
# 고객별 구매 빈도 확인 위한 pivoting
df_pivot2 = df0.pivot_table(index = 'Customer_ID', values = 'Product_No', aggfunc = 'count')

In [9]:
# IQR 기준 이상치 판단(Q3 + (Q3 - Q1)*1.5)
df_pivot2['Product_No'].describe()

count    41147.000000
mean         2.430311
std          3.851730
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max        402.000000
Name: Product_No, dtype: float64

In [12]:
# 2번 이상 구매 고객 sorting, 이상치 제거
member_index2 = df_pivot2[(df_pivot2['Product_No'] > 1)&(df_pivot2['Product_No'] < 5)].index
df = df0[df0['Customer_ID'].isin(member_index2)]

In [13]:
# CTG, Sub_CTG 인코딩 및 datetime data numeric으로 변환
encoder1 = LabelEncoder()
encoder2 = LabelEncoder()

df.loc[:, 'CTG'] = encoder1.fit_transform(df['CTG'])

df.loc[:, 'Sub_CTG'] = encoder2.fit_transform(df['Sub_CTG'])

df.loc[:, 'Order_Date'] = pd.to_numeric(df['Order_Date'])

C:\Users\J13868\AppData\Local\Temp\ipykernel_19688\1267521601.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'CTG'] = encoder1.fit_transform(df['CTG'])
C:\Users\J13868\AppData\Local\Temp\ipykernel_19688\1267521601.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'CTG'] = encoder1.fit_transform(df['CTG'])
C:\Users\J13868\AppData\Local\Temp\ipykernel_19688\1267521601.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [14]:
# 각 고객의 구매 이력을 시계열 데이터로 변환
X = []
y = []
member_ids = []
maxlen = 0
for member_id in df['Customer_ID'].unique():
    member_df = df[df['Customer_ID'] == member_id].sort_values('Order_Date')
    for i in range(len(member_df) - 1):
        X.append(member_df.iloc[:i+1][['Product_No', 'CTG', 'Sub_CTG' ]].values)
        y.append(member_df.iloc[i+1]['CTG'])
        member_ids.append(member_id)
        maxlen = max(maxlen, i+1)

# 시계열 데이터를 동일한 길이로 패딩
X = pad_sequences(X, maxlen=maxlen, dtype='float32')

X[np.isnan(X)] = 0

In [15]:
# 데이터를 학습/테스트 세트로 분리
X_train, X_test, y_train, y_test, member_ids_train, member_ids_test = train_test_split(X, y, member_ids, test_size=0.2, random_state=0)

# LightGBM 모델 학습
model = LGBMClassifier(num_leaves=56,
                       min_data_in_leaf=30,
                       max_depth=7,
                       bagging_fraction=0.6,
                       feature_fraction=0.75,
                       random_state=0)
model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6


LGBMClassifier(bagging_fraction=0.6, feature_fraction=0.75, max_depth=7,
               min_data_in_leaf=30, num_leaves=56, random_state=0)

In [16]:
# 모델 평가
y_pred = model.predict(X_test.reshape(X_test.shape[0], -1))

last_purchase = {}
for member_id in df['Customer_ID'].unique():
    member_df = df[df['Customer_ID'] == member_id].sort_values('Order_Date')
    last_purchase_date = member_df.iloc[-1]['Order_Date']
    last_purchase_products = member_df[member_df['Order_Date'] == last_purchase_date]['CTG'].unique()
    last_purchase[member_id] = last_purchase_products

correct = []
for i in range(len(y_pred)):
    if y_pred[i] in last_purchase[member_ids_test[i]]:
        correct.append(1)
    else:
        correct.append(0)

# 정확도 계산
accuracy = sum(correct) / len(correct)
print('Accuracy Score:', accuracy)

Accuracy Score: 0.8630683790350573


In [17]:
results_df = pd.DataFrame(columns=['Customer_ID', 'Predicted_CTG'])

for member_id in df['Customer_ID'].unique():
    # Get the data for the current MemberID
    member_df = df[df['Customer_ID'] == member_id].sort_values('Order_Date')
    
    X2 = member_df[['Product_No','CTG', 'Sub_CTG']].values
    X2 = pad_sequences(X2[np.newaxis,:,:], maxlen=maxlen, dtype='float32')
    X2[np.isnan(X2)] = 0
    X2 = X2.reshape(1, -1)
    
    y_pred2 = model.predict(X2)
    
    results_df = pd.concat([results_df, pd.DataFrame({'Customer_ID': [member_id], 'Predicted_CTG': [y_pred2[0]]})], ignore_index=True)

In [18]:
Sector_df = pd.DataFrame(columns=['Cutomer_ID', 'A', 'B', 'C', 'D', 'E'])

for member_id in df['Customer_ID'].unique():
    # Get the purchase history for the current customer
    member_df2 = df[df['Customer_ID'] == member_id].sort_values('Order_Date')
    
    # 섹터별 구매 빈도 확인
    sector_counts = member_df2['Sector'].value_counts()
    
    Sector_df = pd.concat([Sector_df, pd.DataFrame({
        'Customer_ID': [member_id],
        'A': [sector_counts.get('A', 0)],
        'B': [sector_counts.get('B', 0)],
        'C': [sector_counts.get('C', 0)],
        'D': [sector_counts.get('D', 0)],
        'E': [sector_counts.get('E', 0)]
    })], ignore_index=True)

In [19]:
summary_df = pd.merge(Sector_df,results_df, on = 'Customer_ID', how = 'inner')

In [20]:
summary_df['Predicted_CTG'] = summary_df['Predicted_CTG'].astype(int)
summary_df['Predicted_CTG'] = encoder1.inverse_transform(summary_df['Predicted_CTG'])

In [21]:
summary_df

,Cutomer_ID,A,B,C,D,E,Customer_ID,Predicted_CTG
0,NaN,0,0,0,2,0,861344.0,DA
1,NaN,0,0,0,2,0,383657.0,DA
2,NaN,0,0,0,4,0,964428.0,DA
3,NaN,0,0,0,3,0,792101.0,DA
4,NaN,0,0,1,1,0,438720.0,DA
...,...,...,...,...,...,...,...,...
18438,NaN,0,0,0,2,0,230539.0,DA
18439,NaN,0,0,0,3,0,809032.0,DA
18440,NaN,0,0,0,4,0,331515.0,DA
18441,NaN,0,0,2,0,0,444315.0,CA


In [ ]:
# Bayes Search 통한 Hyper Parameter Fine Tuning

from skopt import BayesSearchCV
from skopt.space import Real, Integer

param_grid = {
    'num_leaves': Integer(7, 63),
    'min_data_in_leaf': Integer(1, 30),
    'max_depth': Integer(-1, 7),
    'bagging_fraction': Real(0.6, 0.9),
    'feature_fraction': Real(0.6, 0.9)
}

# 베이지안 최적화
model = LGBMClassifier(random_state=0)
bayes_search = BayesSearchCV(model, param_grid, cv=5)
bayes_search.fit(X_train.reshape(X_train.shape[0], -1), y_train)

best_params = bayes_search.best_params_
print('Best parameters:', best_params)

# LightGBM 모델 학습
model = LGBMClassifier(**best_params)
model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 모델 평가
y_pred = model.predict(X_test.reshape(X_test.shape[0], -1))
print('Accuracy:', accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

# Confusion Matrix 계산
cm = confusion_matrix(y_test, y_pred)

# Confusion Matrix DataFrame으로 변환
cm_df = pd.DataFrame(cm, columns=[f'Predicted {c}' for c in encoder1.classes_], index=[f'Actual {c}' for c in encoder1.classes_])

error_df = cm_df.stack().reset_index()
error_df.columns = ['Actual Class', 'Predicted Class', 'Count']
error_df = error_df[error_df['Actual Class'] != error_df['Predicted Class']].sort_values('Count', ascending=False)

error_df['Actual Class'] = error_df['Actual Class'].str.replace('Actual ', '')
error_df['Predicted Class'] = error_df['Predicted Class'].str.replace('Predicted ', '')


error_df